# Test point-based registration

In [1]:
%matplotlib widget
import numpy as np
import os
import open3d as o3d
import copy
import time

import planeslam.io as io

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Read in airsim LiDAR and pose data

In [2]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

In [3]:
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_20_samples_1', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

Open3D ICP registration

In [4]:
# Visualization helper
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

In [11]:
# Initialize source and target
source = o3d.geometry.PointCloud()
source.points = o3d.utility.Vector3dVector(PC_data[0])
source.estimate_normals()
source.orient_normals_towards_camera_location()

target = o3d.geometry.PointCloud()
target.points = o3d.utility.Vector3dVector(PC_data[1])
target.estimate_normals()
target.orient_normals_towards_camera_location()

In [6]:
# Visualize
o3d.visualization.draw_geometries([target])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [7]:
# Initial guess transform 
trans_init = np.eye(4)
threshold = 0.02

evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, trans_init)
print(evaluation)

RegistrationResult with fitness=4.595588e-04, inlier_rmse=1.658737e-02, and correspondence_set size of 3
Access transformation to get result.


In [8]:
# Point-to-point ICP
print("Apply point-to-point ICP")
start_time = time.time()
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print("Elapased time: ", time.time() - start_time)
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

Apply point-to-point ICP
Elapased time:  0.12083935737609863
RegistrationResult with fitness=6.280637e-03, inlier_rmse=1.329315e-02, and correspondence_set size of 41
Access transformation to get result.
Transformation is:
[[ 9.62607006e-01  7.05381633e-05  2.70901728e-01 -9.08054546e-01]
 [-1.56330597e-04  9.99999944e-01  2.95113544e-04 -6.75400308e-03]
 [-2.70901692e-01 -3.26428594e-04  9.62606964e-01 -4.43856240e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [9]:
demo_icp_pcds = o3d.data.DemoICPPointClouds()
source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.io.read_point_cloud(demo_icp_pcds.paths[1])

np.asarray(source.normals)

[Open3D INFO] Downloading https://github.com/isl-org/open3d_downloads/releases/download/20220201-data/DemoICPPointClouds.zip
[Open3D INFO] Downloaded to C:\Users\adamd/open3d_data/download/DemoICPPointClouds/DemoICPPointClouds.zip
[Open3D INFO] Extracting C:\Users\adamd/open3d_data/download/DemoICPPointClouds/DemoICPPointClouds.zip.
[Open3D INFO] Extracted to C:\Users\adamd/open3d_data/extract/DemoICPPointClouds.


array([[-0.62956297,  0.66870427, -0.39558202],
       [ 0.01322357,  0.06152465, -0.99801797],
       [ 0.01659746,  0.06297147, -0.99787724],
       ...,
       [-0.24384584, -0.52947396, -0.81252468],
       [-0.05902789, -0.70068824, -0.71102154],
       [-0.15078427, -0.55733532, -0.81648111]])

In [12]:
print("Apply point-to-plane ICP")
start_time = time.time()
reg_p2l = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())
print("Elapased time: ", time.time() - start_time)
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
draw_registration_result(source, target, reg_p2l.transformation)

Apply point-to-plane ICP
Elapased time:  0.05086207389831543
RegistrationResult with fitness=0.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 0
Access transformation to get result.
Transformation is:
[[ 9.99986532e-01  3.46789296e-03 -3.86133754e-03 -2.71526439e-02]
 [-3.44794998e-03  9.99980745e-01  5.15951917e-03 -8.04419914e+05]
 [ 3.87915585e-03 -5.14613598e-03  9.99979235e-01  2.26548884e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
